In [34]:
from glob import glob
import os
import shutil
import random

In [78]:
# labels3 = glob('../../datasets/aihub_safety/val/5.전체/*.json')
labels1 = glob('../../datasets/aihub_safety/val/*/*/*/*.json')
labels2 = glob('../../datasets/aihub_safety/val/*/*/*.json')
labels = labels1 + labels2 
# images1 = glob('../../datasets/aihub_safety/val/*/*.jpg')
# images2 = glob('../../datasets/aihub_safety/val/*/*/*.jpg')
# images = images1 + images2
imgs = [i.replace('라벨','원천').replace('/5.전체','').replace('json', 'jpg') for i in labels]
a = set()

for i in labels:
    a.add(os.path.dirname(i))
a = list(a)

In [83]:
a.sort()
a

['../../datasets/aihub_safety/val/[라벨]1.공동주택/고양힐스테이트/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/고양힐스테이트/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/동탄/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/동탄/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/동탄/5.전체',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/5.전체',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/성남 금빛매종 공동주택 신축공사/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/성남 금빛매종 공동주택 신축공사/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/1.안전보호구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/2.중장비만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/5.전체',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/주안 재건축/2.중장비만',
 '../../da

In [76]:
for label in labels:
    if os.path.basename(label) == 'S2-N0509M00883.json':
        print(label)

In [77]:
print(len(imgs), len(labels))

56377 56377


In [80]:
imgs[0]

'../../datasets/aihub_safety/val/[원천]6.상업시설/송도동/S2-N2001M40855.jpg'

In [81]:
random.seed(42)
random.shuffle(a)

In [82]:
a

['../../datasets/aihub_safety/val/[라벨]9.주차장_고양스타필드/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]4.공항시설/인천공항 제2여객터미널 증축공사(동측)/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]5.물류센터/인천공항 화물터미널E/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]3.공장_가산동 A1타워 지식산업 센터/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/동탄/5.전체',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/주안 재건축/2.중장비만',
 '../../datasets/aihub_safety/val/[라벨]7.스튜디오/스튜디오/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]6.상업시설/신사동 복합 시설/3.구조물_비계만',
 '../../datasets/aihub_safety/val/[라벨]4.공항시설/인천공항 제2여객터미널 증축공사(서측)/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/2.중장비만',
 '../../datasets/aihub_safety/val/[라벨]4.공항시설/인천공항 제2여객터미널 증축공사(서측)/5.전체x',
 '../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/2.중장비만',
 '../../datasets/aihub_safety/val/[라벨]3.공장_가산동 A1타워 지식산업 센터/4.자재및공구만',
 '../../datasets/aihub_safety/val/[라벨]4.공항시설/인천공항 제2여객터미널 증축공사(동측)/4.자재및공구만',
 '../../datasets/aihub_safety/val/

In [87]:
import json
import cv2
import numpy as np
import shutil
import random

def coord_organize(box):
    return [min(box[0],box[2]), min(box[1],box[3]), max(box[0],box[2]), max(box[1],box[3])]

def cvt_yoloformat(xyxy,w,h):
    box_w = xyxy[2] - xyxy[0]
    box_h = xyxy[3] - xyxy[1]
    return [(xyxy[0] + box_w/2)/w ,(xyxy[1] + box_h/2)/h, box_w/w, box_h/h ]


def json_transform():
    des_path = '../../datasets/aihub_safety/valid'
    count = 0
    f_count = 0

    for i in a:
        if(os.path.basename(i)[0] not in ['5','2','1']):
            # if(os.path.basename(i)[0] not in ['2','1']):
            continue
        lbls = glob(i.replace('[','[[]') + '/*')
        for j in lbls:
            f_count += 1
            # print(os.path.dirname(j).split('/')[-1][0] )
            # if(os.path.dirname(j).split('/')[-1][0] == '2'):
            #     # print(j.replace('2.중장비만','5.전체'), os.path.exists(j.replace('2.중장비만','5.전체')))
            #     if os.path.exists(j.replace('2.중장비만','5.전체')):
            #         print('change',j,' @ ',j.replace('2.중장비만','5.전체'))
            #         j = j.replace('2.중장비만','5.전체')
            # img_f = glob('../../datasets/aihub_safety/val/[[]원천]' + j.split('[라벨]')[1][0] + '*/' + os.path.basename(j)[:-4] +'jpg')
            if os.path.dirname(j).split('/')[-1] == '5.전체x':
                continue
            img_f = j.replace('라벨','원천').replace('/5.전체','').replace('json', 'jpg').replace('/1.안전보호구만','').replace('/2.중장비만','')
            # print('../../datasets/aihub_safety/val/[[]원천]' + j.split('[라벨]')[1][0] + '*/' + os.path.basename(j)[:-4] +'jpg')
            try:
                img = cv2.imread(img_f)
            except Exception as e:
                print(e)
                print('j : ',j,'img_f : ',img_f)
            # print(img)
            w, h = img.shape[1] , img.shape[0]

            try:
                with open(j, 'r',encoding='utf-8-sig') as f:
                    data = json.load(f)
                    annos = data['annotations']
            except Exception as e:
                print(e)
                try:
                    with open(j, 'r') as f:
                        data = json.load(f)
                        annos = data['annotations']
                except Exception as e:
                    print(e, '\n json data 읽기 실패')
                    continue
            

            

            data_extst = False
            new_datas = []
            
            if len(annos):
                for anno in annos:
                    try:
                        cls = int(anno['class'] )
                        box = anno['box']
                    except:
                        cls = None
                        box = []
                    # print('box : ', box, 'cls : ', cls)
                    # print((len(box)) and (int(cls) < 20))
                    if (len(box)) and (int(cls) < 20):
                        data_extst = True

                        box = coord_organize(box)
                        box = cvt_yoloformat(box,w,h)
                        new_data = str(int(cls) - 1) + ' ' + ' '.join(str(k) for k in box)  + '\n'
            
                        new_datas.append(new_data)

                if(data_extst):
                    count += 1
                    try:

                        shutil.copy(img_f , des_path + '/images') # image copy
                        with open(des_path + '/labels/' + os.path.basename(j)[:-4] + 'txt', 'w') as f: # write converted labeled file
                            for p in new_datas:
                                f.write(p)
                    except:
                        print('failed to write files or can\'t find image file', img_f,j)
            if f_count % 500 == 0 : print(f'f_count : {f_count}, count : {count}, folder : {i}' ) 
json_transform()

f_count : 500, count : 472, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/5.전체
f_count : 1000, count : 954, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/5.전체
f_count : 1500, count : 1431, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/서대문/5.전체
f_count : 2000, count : 1925, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/2.중장비만
f_count : 2500, count : 2359, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/5.전체
f_count : 3000, count : 2731, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/송도호반/5.전체
f_count : 3500, count : 3141, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/1.안전보호구만
f_count : 4000, count : 3641, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/1.안전보호구만
f_count : 4500, count : 4141, folder : ../../datasets/aihub_safety/val/[라벨]1.공동주택/하남감일 A-7BL 11공구 아파트 신축공사/2.중장비만
f_count : 11000, count : 4332, folder : ../../datasets/aihub_safety/val/[라벨]3.공장_가산동 A1타워 지식산업 센터/5.전체
f_c

In [49]:
len(labels)

9627

In [11]:
len(images)

1657489

In [12]:
## images 와 labels 폴더 포멧인 경우 random 42로 샘플링 

def sampling(img_path,label_path, des_path, size):
    if img_path[-1] == '/':
        img_path = img_path[:-1]
    if label_path[-1] == '/':
        label_path = label_path[:-1]
    if des_path[-1] == '/':
        des_path = des_path[:-1]
        
    img_source_dir = img_path
    label_source_dir = label_path
    imgs = sorted(glob(img_source_dir + '/*'))
    # print(imgs[0].replace('images','labels')[:-3] + 'txt')
    # return 0
    labels = [i.replace('images','labels')[:-3] + 'txt' for i in imgs]
    # labels = sorted(glob(label_source_dir + '/*'))
    img_des_path = des_path + '/sampled_images'
    label_des_path = des_path + '/sampled_labels'
    if not os.path.exists(img_des_path):
        os.makedirs(img_des_path)
        print(1)
    if not os.path.exists(label_des_path):
        os.makedirs(label_des_path)
    
    random.seed(42)
    random.shuffle(imgs)
    random.seed(42)
    random.shuffle(labels)

    print(imgs[:3])
    print(labels[:3])
    img_size = int(size * len(imgs))
    label_size = int(size * len(labels))
    imgs = imgs[:img_size]
    labels = labels[:label_size]

    for i in imgs:
        shutil.copy(i,img_des_path)
    for i in labels:
        shutil.copy(i,label_des_path)
    

In [13]:
ratio = 100000 / len(glob('../../datasets/aihub_safety/human_car/images/*'))


In [14]:
ratio

0.225392746861406

In [15]:
os.path.exists('../../datasets/aihub_safety/human_car/labels/S2-N0002M00001.txt')

True

In [16]:
sampling('../../datasets/aihub_safety/human_car/images','../../datasets/aihub_safety/human_car/labels','../../datasets/aihub_safety/sampled_human_car',ratio)

1
['../../datasets/aihub_safety/human_car/images/S2-N1106M07362.jpg', '../../datasets/aihub_safety/human_car/images/S2-N1105M07773.jpg', '../../datasets/aihub_safety/human_car/images/S2-N1802M07061.jpg']
['../../datasets/aihub_safety/human_car/labels/S2-N1106M07362.txt', '../../datasets/aihub_safety/human_car/labels/S2-N1105M07773.txt', '../../datasets/aihub_safety/human_car/labels/S2-N1802M07061.txt']


In [17]:
## ratio

def train_valid_test_split(img_path,label_path,des_path,ratio):
    if img_path[-1] == '/':
        img_path = img_path[:-1]
    if label_path[-1] == '/':
        label_path = label_path[:-1]
    if des_path[-1] == '/':
        des_path = des_path[:-1]

    imgs = sorted(glob(img_path + '/*'))
    random.seed(42)
    random.shuffle(imgs)
    # labels = [label_path + '/' + os.path.basename(i)[:-3] + 'txt' for i in imgs]
    labels = sorted(glob(label_path + '/*'))

    list01 = ['train', 'valid', 'test']

    for dirname in list01:
        if not os.path.exists(des_path + '/' + dirname):
            os.mkdir(des_path + '/' + dirname)
    for dirname in list01:
        for dirname2 in ['images', 'labels']:
            print(dirname2)
            if not os.path.exists(des_path + '/' + dirname + '/' + dirname2):
                os.mkdir(des_path + '/' + dirname + '/' + dirname2)

    size = len(imgs)

    

    # random.seed(42)
    # random.shuffle(labels)

    ratio_sum = 0
    for i in ratio:
        ratio_sum += i

    list02 = []
    for idx, ratio in enumerate(ratio):
        
        if len(list02):        
            list02.append(int(ratio / ratio_sum * len(imgs)) + list02[-1])
        else:
            list02.append(int(ratio / ratio_sum * len(imgs)))

        if len(list02) == 2:
            temp_imgs = imgs[list02[-2]:list02[-1]]
            temp_labels = labels[list02[-2]:list02[-1]]

        elif len(list02) == 3:
            temp_imgs = imgs[list02[-2]:]
            temp_labels = labels[list02[-2]:]
            print('test')
        else:
            temp_imgs = imgs[:list02[-1]]
            temp_labels = labels[:list02[-1]]

        for i in range(len(temp_imgs)):
            # print(temp_imgs[i])
            # print(temp_labels[i])
            
            try:
                shutil.copy(temp_labels[i],des_path + '/' + list01[idx] + '/labels')
                shutil.copy(temp_imgs[i],des_path + '/' + list01[idx] + '/images')
            except:
                pass
        
    print(list02)



In [18]:
train_valid_test_split('../../datasets/aihub_safety/sampled_human_car/sampled_images','../../datasets/aihub_safety/sampled_human_car/sampled_labels','../../datasets/aihub_safety/sampled_human_car',[17,2,1])

images
labels
images
labels
images
labels
test
[85000, 95000, 100000]
